In [1]:
pip install nba_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 13.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for numpy: filename=numpy-1.26.4-cp313-cp313-macosx_14_0_x86_64.whl size=5923090 sha256=1d12a7ddda95c7ba4173b54fd837d4c82c06d60c3b5afa538081036f9c15d5f6
  Stored in directory: /Users/mattkukulich/Library/Caches/pip/wheels/8b/2d/9f/b6b46373f328e2ef50388915d351ccacbedac929459b5459bf
Successfully built numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.2
    Uninstalling numpy-2.2.2:
      Successfully uninstalled numpy-2.2.2
Note: you may need to restart the kernel to use updated packages.


In [27]:
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import LeagueGameLog, PlayerGameLog
from nba_api.stats.static import players
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

class NBAPlayerPropsPredictor:
    def __init__(self):
        self.scaler = StandardScaler()
        self.model = RandomForestRegressor(n_estimators=100, random_state=42)
        
    def get_player_games(self, player_id, season='2024-25'):
        """Fetch player game logs for a given season"""
        game_logs = PlayerGameLog(player_id=player_id, season=season)
        df = game_logs.get_data_frames()[0]
        # Convert GAME_DATE to datetime
        df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
        # Sort by date to ensure correct order
        df = df.sort_values('GAME_DATE')
        return df
    
    def prepare_features(self, df):
        """Prepare features for the model"""
        # Calculate rolling averages for key statistics
        feature_columns = ['PTS', 'MIN', 'FGA', 'FG_PCT', 'FG3A', 'FG3_PCT', 
                         'FTA', 'FT_PCT', 'REB', 'AST', 'TOV']
        
        # Create rolling averages for last 5, 10 games
        for col in feature_columns:
            df[f'{col}_rolling_5'] = df[col].rolling(window=5, min_periods=1).mean()
            df[f'{col}_rolling_10'] = df[col].rolling(window=10, min_periods=1).mean()
        
        # Drop rows with missing values
        df = df.dropna()
        
        return df
    
    def train_model(self, df, target_stat='PTS'):
        """Train the model for a specific statistic"""
        # Prepare features
        feature_cols = [col for col in df.columns if '_rolling_' in col]
        X = df[feature_cols]
        y = df[target_stat]
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # Train model
        self.model.fit(X_train_scaled, y_train)
        
        # Calculate accuracy
        train_score = self.model.score(X_train_scaled, y_train)
        test_score = self.model.score(X_test_scaled, y_test)
        
        return train_score, test_score
    
    def predict_next_game(self, df, target_stat='PTS'):
        """Predict stats for next game"""
        # Prepare features for the most recent game
        feature_cols = [col for col in df.columns if '_rolling_' in col]
        latest_features = df[feature_cols].iloc[-1:]
        
        # Scale features
        latest_features_scaled = self.scaler.transform(latest_features)
        
        # Make prediction
        prediction = self.model.predict(latest_features_scaled)[0]
        
        return prediction, df['GAME_DATE'].iloc[-1]

def find_player_id(name):
    """Find player ID by name"""
    nba_players = players.get_players()
    player_matches = [player for player in nba_players 
                     if name.lower() in player['full_name'].lower()]
    return player_matches

def main():
    predictor = NBAPlayerPropsPredictor()
    
    while True:
        print("\nNBA Player Props Predictor")
        print("-------------------------")
        print("1. Search player by name")
        print("2. Enter player ID directly")
        print("3. Exit")
        
        choice = input("Enter your choice (1-3): ")
        
        if choice == '1':
            name = input("\nEnter player name (or part of name): ")
            matches = find_player_id(name)
            
            if not matches:
                print("No players found with that name.")
                continue
                
            print("\nMatching players:")
            for i, player in enumerate(matches, 1):
                print(f"{i}. {player['full_name']} (ID: {player['id']})")
            
            if len(matches) > 1:
                try:
                    selection = int(input("\nSelect player number: ")) - 1
                    player_id = matches[selection]['id']
                except (ValueError, IndexError):
                    print("Invalid selection.")
                    continue
            else:
                player_id = matches[0]['id']
                
        elif choice == '2':
            try:
                player_id = int(input("\nEnter player ID: "))
            except ValueError:
                print("Invalid player ID. Please enter a number.")
                continue
                
        elif choice == '3':
            print("Goodbye!")
            break
            
        else:
            print("Invalid choice. Please try again.")
            continue
            
        try:
            print(f"\nFetching data...")
            # Get game logs
            df = predictor.get_player_games(player_id)
            
            print("\nLast 5 games:")
            last_5_games = df[['GAME_DATE', 'MATCHUP', 'PTS']].tail()
            print(last_5_games.to_string())
            
            # Prepare features
            df_prepared = predictor.prepare_features(df)
            
            # Train model for points
            train_score, test_score = predictor.train_model(df_prepared, target_stat='PTS')
            print(f"\nTraining Score: {train_score:.3f}")
            print(f"Testing Score: {test_score:.3f}")
            
            # Predict next game
            prediction, last_game_date = predictor.predict_next_game(df_prepared, target_stat='PTS')
            print(f"\nPredicting for the next game after: {last_game_date.strftime('%Y-%m-%d')}")
            print(f"Predicted points: {prediction:.1f}")
            
            input("\nPress Enter to continue...")
            
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            print("Please check if the player ID is valid and the NBA API is accessible.")

if __name__ == "__main__":
    main()


NBA Player Props Predictor
-------------------------
1. Search player by name
2. Enter player ID directly
3. Exit

Matching players:
1. Dell Curry (ID: 209)
2. Eddy Curry (ID: 2201)
3. JamesOn Curry (ID: 201191)
4. Michael Curry (ID: 688)
5. Seth Curry (ID: 203552)
6. Stephen Curry (ID: 201939)
7. Carey Scurry (ID: 78102)

Fetching data...

Last 5 games:
   GAME_DATE      MATCHUP  PTS
4 2025-01-23  GSW vs. CHI   21
3 2025-01-25  GSW vs. LAL   13
2 2025-01-29  GSW vs. OKC   21
1 2025-01-31  GSW vs. PHX   14
0 2025-02-03  GSW vs. ORL   24

Training Score: 0.834
Testing Score: -0.020

Predicting for the next game after: 2025-02-03
Predicted points: 19.1

NBA Player Props Predictor
-------------------------
1. Search player by name
2. Enter player ID directly
3. Exit

Matching players:
1. Mitchell Anderson (ID: 76043)
2. Mitchell Butler (ID: 348)
3. Mitchell Creek (ID: 1628249)
4. Ajay Mitchell (ID: 1642349)
5. Davion Mitchell (ID: 1630558)
6. Donovan Mitchell (ID: 1628378)
7. Mike Mitche

In [2]:
#Importing libraries
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import LeagueGameLog, PlayerGameLog
from nba_api.stats.static import players
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')


#Making a class called NBAPlayerPropsPredictor
class NBAPlayerPropsPredictor:
    def __init__(self):
        self.scalers = {}
        self.models = {}
        self.target_stats = ['PTS', 'REB', 'AST']
        
        # Initialize a model and scaler for each target statistic
        for stat in self.target_stats:
            self.scalers[stat] = StandardScaler()
            self.models[stat] = RandomForestRegressor(n_estimators=100, random_state=42)

    #fetching player game logs  
    def get_player_games(self, player_id, season='2024-25'):
        """Fetch player game logs for a given season"""
        game_logs = PlayerGameLog(player_id=player_id, season=season)
        df = game_logs.get_data_frames()[0]
        # Convert GAME_DATE to datetime
        df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
        # Sort by date to ensure correct order
        df = df.sort_values('GAME_DATE')
        return df
    
    #feature engineering
    def prepare_features(self, df):
        """Prepare features for the model"""
        # Calculate rolling averages for key statistics
        feature_columns = ['PTS', 'MIN', 'FGA', 'FG_PCT', 'FG3A', 'FG3_PCT', 
                         'FTA', 'FT_PCT', 'REB', 'AST', 'TOV']
        
        # Create rolling averages for last 5, 10 games
        for col in feature_columns:
            df[f'{col}_rolling_5'] = df[col].rolling(window=5, min_periods=1).mean()
            df[f'{col}_rolling_10'] = df[col].rolling(window=10, min_periods=1).mean()
        
        # Drop rows with missing values
        df = df.dropna()
        
        return df
    
    #training the model using Random Forest
    def train_model(self, df):
        """Train models for all target statistics"""
        results = {}
        feature_cols = [col for col in df.columns if '_rolling_' in col]
        
        for target_stat in self.target_stats:
            # Prepare features and target
            X = df[feature_cols]
            y = df[target_stat]
            
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            
            # Scale features
            X_train_scaled = self.scalers[target_stat].fit_transform(X_train)
            X_test_scaled = self.scalers[target_stat].transform(X_test)
            
            # Train model
            self.models[target_stat].fit(X_train_scaled, y_train)
            
            # Calculate accuracy
            train_score = self.models[target_stat].score(X_train_scaled, y_train)
            test_score = self.models[target_stat].score(X_test_scaled, y_test)
            
            results[target_stat] = (train_score, test_score)
        
        return results
    
    #predicting the next game
    def predict_next_game(self, df):
        """Predict all stats for next game"""
        predictions = {}
        feature_cols = [col for col in df.columns if '_rolling_' in col]
        latest_features = df[feature_cols].iloc[-1:]
        
        for target_stat in self.target_stats:
            # Scale features
            latest_features_scaled = self.scalers[target_stat].transform(latest_features)
            
            # Make prediction
            prediction = self.models[target_stat].predict(latest_features_scaled)[0]
            predictions[target_stat] = prediction
        
        return predictions, df['GAME_DATE'].iloc[-1]

#searching for players
def find_player_id(name):
    """Find player ID by name"""
    nba_players = players.get_players()
    player_matches = [player for player in nba_players 
                     if name.lower() in player['full_name'].lower()]
    return player_matches

#Main program loop
def main():
    predictor = NBAPlayerPropsPredictor()
    
    while True:
        print("\nNBA Player Props Predictor")
        print("-------------------------")
        print("1. Search player by name")
        print("2. Enter player ID directly")
        print("3. Exit")
        
        choice = input("Enter your choice (1-3): ")
        
        #handling user input
        if choice == '1':
            name = input("\nEnter player name (or part of name): ")
            matches = find_player_id(name)
            
        #fetching data and making predictions
            if not matches:
                print("No players found with that name.")
                continue
                
            print("\nMatching players:")
            for i, player in enumerate(matches, 1):
                print(f"{i}. {player['full_name']} (ID: {player['id']})")
            
            if len(matches) > 1:
                try:
                    selection = int(input("\nSelect player number: ")) - 1
                    player_id = matches[selection]['id']
                except (ValueError, IndexError):
                    print("Invalid selection.")
                    continue
            else:
                player_id = matches[0]['id']
                
        elif choice == '2':
            try:
                player_id = int(input("\nEnter player ID: "))
            except ValueError:
                print("Invalid player ID. Please enter a number.")
                continue
                
        elif choice == '3':
            print("Goodbye!")
            break
            
        else:
            print("Invalid choice. Please try again.")
            continue
            
        try:
            print(f"\nFetching data...")
            # Get game logs
            df = predictor.get_player_games(player_id)
            
            print("\nLast 5 games:")
            last_5_games = df[['GAME_DATE', 'MATCHUP', 'PTS', 'REB', 'AST']].tail()
            print(last_5_games.to_string())
            
            # Prepare features
            df_prepared = predictor.prepare_features(df)
            
            # Train models for all stats
            training_results = predictor.train_model(df_prepared)
            
            print("\nModel Performance:")
            for stat, (train_score, test_score) in training_results.items():
                print(f"\n{stat}:")
                print(f"Training Score: {train_score:.3f}")
                print(f"Testing Score: {test_score:.3f}")
            
            # Predict next game
            predictions, last_game_date = predictor.predict_next_game(df_prepared)
            print(f"\nPredicting for the next game after: {last_game_date.strftime('%Y-%m-%d')}")
            for stat, prediction in predictions.items():
                print(f"Predicted {stat}: {prediction:.1f}")
            
            input("\nPress Enter to continue...")
            
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            print("Please check if the player ID is valid and the NBA API is accessible.")

if __name__ == "__main__":
    main()


NBA Player Props Predictor
-------------------------
1. Search player by name
2. Enter player ID directly
3. Exit

Matching players:
1. Josh Giddey (ID: 1630581)

Fetching data...

Last 5 games:
   GAME_DATE      MATCHUP  PTS  REB  AST
4 2025-01-27  CHI vs. DEN   12    4   10
3 2025-01-29    CHI @ BOS   10    4    6
2 2025-01-31    CHI @ TOR   15    7    6
1 2025-02-02    CHI @ DET   20   10    4
0 2025-02-04  CHI vs. MIA   24    5    7

Model Performance:

PTS:
Training Score: 0.871
Testing Score: -0.155

REB:
Training Score: 0.833
Testing Score: 0.289

AST:
Training Score: 0.867
Testing Score: -0.708

Predicting for the next game after: 2025-02-04
Predicted PTS: 20.2
Predicted REB: 6.5
Predicted AST: 6.6

NBA Player Props Predictor
-------------------------
1. Search player by name
2. Enter player ID directly
3. Exit
Goodbye!


In [18]:
#Importing libraries
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import LeagueGameLog, PlayerGameLog
from nba_api.stats.static import players
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')


#Making a class called NBAPlayerPropsPredictor
class NBAPlayerPropsPredictor:
    def __init__(self):
        self.scalers = {}
        self.models = {}
        self.target_stats = ['PTS', 'REB', 'AST']
        
        # Initialize a model and scaler for each target statistic
        for stat in self.target_stats:
            self.scalers[stat] = StandardScaler()
            self.models[stat] = RandomForestRegressor(n_estimators=100, random_state=42)

    #fetching player game logs  
    def get_player_games(self, player_id, season='2024-25'):
        """Fetch player game logs for a given season"""
        game_logs = PlayerGameLog(player_id=player_id, season=season)
        df = game_logs.get_data_frames()[0]
        # Convert GAME_DATE to datetime
        df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
        # Sort by date to ensure correct order
        df = df.sort_values('GAME_DATE')
        return df
    
    #fetching team game logs (defensive data)
    def get_team_games(self, season='2024-25'):
        """Fetch league-wide team game logs for a given season"""
        team_game_logs = LeagueGameLog(season=season)
        df = team_game_logs.get_data_frames()[0]
        return df
    
    #feature engineering
    def prepare_features(self, df, team_df):
        """Prepare features for the model, including team defensive data"""
        # Calculate rolling averages for key statistics
        feature_columns = ['PTS', 'MIN', 'FGA', 'FG_PCT', 'FG3A', 'FG3_PCT', 
                         'FTA', 'FT_PCT', 'REB', 'AST', 'TOV']
        
        # Create rolling averages for last 5, 10 games
        for col in feature_columns:
            df[f'{col}_rolling_5'] = df[col].rolling(window=5, min_periods=1).mean()
            df[f'{col}_rolling_10'] = df[col].rolling(window=10, min_periods=1).mean()
        
        #Convert GAME_DATE in team_df to datetime
        team_df['GAME_DATE'] = pd.to_datetime(team_df['GAME_DATE'])

        #Convert GAME_DATE in player gamelog to datetime
        df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
        
        #Merge with team defensive data
        df = pd.merge(df, team_df[['GAME_DATE', 'MATCHUP', 'PTS', 'AST', 'REB']], on='GAME_DATE', how='left', suffixes=('', '_team'))
        
        # Drop rows with missing values
        df = df.dropna()
        
        return df
    
    #training the model using Random Forest
    def train_model(self, df):
        """Train models for all target statistics"""
        results = {}
        feature_cols = [col for col in df.columns if '_rolling_' in col]
        
        for target_stat in self.target_stats:
            # Prepare features and target
            X = df[feature_cols]
            y = df[target_stat]
            
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            
            # Scale features
            X_train_scaled = self.scalers[target_stat].fit_transform(X_train)
            X_test_scaled = self.scalers[target_stat].transform(X_test)
            
            # Train model
            self.models[target_stat].fit(X_train_scaled, y_train)
            
            # Calculate accuracy
            train_score = self.models[target_stat].score(X_train_scaled, y_train)
            test_score = self.models[target_stat].score(X_test_scaled, y_test)
            
            results[target_stat] = (train_score, test_score)
        
        return results
    
    #predicting the next game
    def predict_next_game(self, df):
        """Predict all stats for next game"""
        predictions = {}
        feature_cols = [col for col in df.columns if '_rolling_' in col]
        latest_features = df[feature_cols].iloc[-1:]
        
        for target_stat in self.target_stats:
            # Scale features
            latest_features_scaled = self.scalers[target_stat].transform(latest_features)
            
            # Make prediction
            prediction = self.models[target_stat].predict(latest_features_scaled)[0]
            predictions[target_stat] = prediction
        
        return predictions, df['GAME_DATE'].iloc[-1]

#searching for players
def find_player_id(name):
    """Find player ID by name"""
    nba_players = players.get_players()
    player_matches = [player for player in nba_players 
                     if name.lower() in player['full_name'].lower()]
    return player_matches

#Main program loop
def main():
    predictor = NBAPlayerPropsPredictor()

    #Fetch team defensive data
    team_df = predictor.get_team_games()
    
    while True:
        print("\nNBA Player Props Predictor")
        print("-------------------------")
        print("1. Search player by name")
        print("2. Enter player ID directly")
        print("3. Exit")
        
        choice = input("Enter your choice (1-3): ")
        
        #handling user input
        if choice == '1':
            name = input("\nEnter player name (or part of name): ")
            matches = find_player_id(name)
            
        #fetching data and making predictions
            if not matches:
                print("No players found with that name.")
                continue
                
            print("\nMatching players:")
            for i, player in enumerate(matches, 1):
                print(f"{i}. {player['full_name']} (ID: {player['id']})")
            
            if len(matches) > 1:
                try:
                    selection = int(input("\nSelect player number: ")) - 1
                    player_id = matches[selection]['id']
                except (ValueError, IndexError):
                    print("Invalid selection.")
                    continue
            else:
                player_id = matches[0]['id']
                
        elif choice == '2':
            try:
                player_id = int(input("\nEnter player ID: "))
            except ValueError:
                print("Invalid player ID. Please enter a number.")
                continue
                
        elif choice == '3':
            print("Goodbye!")
            break
            
        else:
            print("Invalid choice. Please try again.")
            continue
            
        try:
            print(f"\nFetching data...")
            # Get player game logs
            df = predictor.get_player_games(player_id)
            
            print("\nLast 5 games:")
            last_5_games = df[['GAME_DATE', 'MATCHUP', 'PTS', 'REB', 'AST']].tail()
            print(last_5_games.to_string())
            
            # Prepare features
            df_prepared = predictor.prepare_features(df, team_df)
            
            # Train models for all stats
            training_results = predictor.train_model(df_prepared)
            
            print("\nModel Performance:")
            for stat, (train_score, test_score) in training_results.items():
                print(f"\n{stat}:")
                print(f"Training Score: {train_score:.3f}")
                print(f"Testing Score: {test_score:.3f}")
            
            # Predict next game
            predictions, last_game_date = predictor.predict_next_game(df_prepared)
            print(f"\nPredicting for the next game after: {last_game_date.strftime('%Y-%m-%d')}")
            for stat, prediction in predictions.items():
                print(f"Predicted {stat}: {prediction:.1f}")
            
            input("\nPress Enter to continue...")
            
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            print("Please check if the player ID is valid and the NBA API is accessible.")

if __name__ == "__main__":
    main()


NBA Player Props Predictor
-------------------------
1. Search player by name
2. Enter player ID directly
3. Exit

Matching players:
1. Shai Gilgeous-Alexander (ID: 1628983)

Fetching data...

Last 5 games:
   GAME_DATE      MATCHUP  PTS  REB  AST
4 2025-01-23  OKC vs. DAL   31    1    7
3 2025-01-26    OKC @ POR   35    5    1
2 2025-01-29    OKC @ GSW   52    3    4
1 2025-02-01  OKC vs. SAC   29    6    9
0 2025-02-03  OKC vs. MIL   34    3    6

Model Performance:

PTS:
Training Score: 1.000
Testing Score: 0.999

REB:
Training Score: 1.000
Testing Score: 0.998

AST:
Training Score: 0.998
Testing Score: 0.993

Predicting for the next game after: 2025-02-03
Predicted PTS: 34.0
Predicted REB: 3.0
Predicted AST: 6.0

NBA Player Props Predictor
-------------------------
1. Search player by name
2. Enter player ID directly
3. Exit
Goodbye!
